# Translate text without profanities

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/translation_to_specific_language.ipynb).

In this example, we will use Guardrails during the translation of a statement from another language to english. We will check whether the translated statement passes the profanity check or not.

## Objective

We want to translate a statement from another languages to English and ensure the translated statement is profanity free.

## Step 0: Setup

In order to run this example, you will need to install `alt-profanity-check` package. You can do so by running the following commands:

In [ ]:
!pip install alt-profanity-check

## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](../rail/output.md).

In this RAIL spec, we:

1. Create an `output` schema that returns a single key-value pair. The key should be 'translated_statement', and the value should be the English translation of the given statement. The translated statement should not have any profanity.

In [ ]:
from profanity_check import predict

In [ ]:
rail_str = """
<rail version="0.1">

<script language='python'>
from profanity_check import predict
from guardrails.validators import Validator, EventDetail, register_validator


from typing import Dict, List


@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    global predict
    global EventDetail
    def validate(self, key, value, schema) -> Dict:
        text = value
        prediction = predict([value])
        if prediction[0] == 1:
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} contains profanity language",
                "",
            )
        return schema
</script>

<output>
    <string
        name="translated_statement"
        description="Translate the given statement into english language"
        format="is-profanity-free"
        on-fail-is-profanity-free="fix" 
    />
</output>


<prompt>

Translate the given statement into english language:

${statement_to_be_translated}

${complete_json_suffix}
</prompt>


</rail>


"""

!!! note

    In order to ensure the translated statement is profanity free, we use `is-profanity-free` as the validator. This validator uses `profanity_check` package.

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [ ]:
import guardrails as gd

from rich import print

guard = gd.Guard.from_rail_string(rail_str)

We see the prompt that will be sent to the LLM:

In [ ]:
print(guard.base_prompt)

Here, `statement_to_be_translated` is the the statement and will be provided by the user at runtime.

## Step 3: Wrap the LLM API call with `Guard`

First, let's try translating a statement that doesn't have any profanity in it.

In [ ]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={"statement_to_be_translated": "quesadilla de pollo"},
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)

print(f"Validated Output: {validated_response}")

We can take a look at the output of the LLM and the validated output using the Guard's internal logs:

In [ ]:
print(guard.state.most_recent_call.tree)

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary). We can see that the output is a dictionary with the correct schema and types.

Next, let's try translating a statement that has profanity in it. We see that the translated statement has been corrected to return an empty string instead of the translated statement.

In [ ]:
raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={"statement_to_be_translated": "убей себя"},
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)

print(f"Validated Output: {validated_response}")

This time around, when we look at the logs, we can see that the output of the LLM was filtered out because it did not pass the profanity check.

In [ ]:
print(guard.state.most_recent_call.tree)